In [1]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, RobertaForSequenceClassification
from transformers import ElectraForPreTraining, ElectraTokenizerFast, AutoTokenizer
small = AutoModelForSequenceClassification.from_pretrained("Seznam/small-e-czech")
robeczech = AutoModelForSequenceClassification.from_pretrained("ufal/robeczech-base")
r_tok = AutoTokenizer.from_pretrained("ufal/robeczech-base")
s_tok = AutoTokenizer.from_pretrained("Seznam/small-e-czech")

Some weights of the model checkpoint at Seznam/small-e-czech were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at Seznam/small-e-czech and are newly initialized: ['classifier.dense.weight', 'classifier.

In [1]:
from models import FineTunedClassifier

In [3]:
classifier = FineTunedClassifier.load_from_checkpoint("projects/Server-Deep-Learning/86w86r5z/checkpoints/epoch=6-step=46111.ckpt", optimizer=None, scheduler=None)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│ ❱ 1 classifier = FineTunedClassifier.load_from_checkpoint("projects/Server-Deep-Learning/86w     │
│   2                                                                                              │
│                                                                                                  │
│ /lnet/aic/personal/kydliceh/non_runable/NLP_venv/lib/python3.10/site-packages/lightning/pytorch/ │
│ core/module.py:1532 in load_from_checkpoint                                                      │
│                                                                                                  │
│   1529 │   │   │   pretrained_model.freeze()                                                     │
│   1530 │   │   │   y_hat = pretrained_model(x)                                                   │
│   1531 │   │   """                                                                               │
│ ❱ 1532 │   │   loaded = _load_from_checkpoint(                                                   │
│   1533 │   │   │   cls,                                                                          │
│   1534 │   │   │   checkpoint_path,                                                              │
│   1535 │   │   │   map_location,                                                                 │
│                                                                                                  │
│ /lnet/aic/personal/kydliceh/non_runable/NLP_venv/lib/python3.10/site-packages/lightning/pytorch/ │
│ core/saving.py:90 in _load_from_checkpoint                                                       │
│                                                                                                  │
│    87 │   if issubclass(cls, pl.LightningDataModule):                                            │
│    88 │   │   return _load_state(cls, checkpoint, **kwargs)                                      │
│    89 │   if issubclass(cls, pl.LightningModule):                                                │
│ ❱  90 │   │   return _load_state(cls, checkpoint, strict=strict, **kwargs)                       │
│    91 │   raise NotImplementedError(f"Unsupported {cls}")                                        │
│    92                                                                                            │
│    93                                                                                            │
│                                                                                                  │
│ /lnet/aic/personal/kydliceh/non_runable/NLP_venv/lib/python3.10/site-packages/lightning/pytorch/ │
│ core/saving.py:136 in _load_state                                                                │
│                                                                                                  │
│   133 │   │   # filter kwargs according to class init unless it allows any argument via kwargs   │
│   134 │   │   _cls_kwargs = {k: v for k, v in _cls_kwargs.items() if k in cls_init_args_name}    │
│   135 │                                                                                          │
│ ❱ 136 │   obj = cls(**_cls_kwargs)                                                               │
│   137 │                                                                                          │
│   138 │   if isinstance(obj, pl.LightningModule):                                                │
│   139 │   │   # give model a chance to load something                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: FineTunedClassifier.__init__() missing 2 required 

In [2]:
dm.prepare_data()

In [11]:
from datasets import load_from_disk

In [23]:
dst = load_from_disk("/home/kydliceh/.cache/LM/512/train")

In [30]:
dst[1:4]["input_ids"].shape

torch.Size([3, 512])

In [34]:
import torch
maximum = 0
minimum = 1000
for x in dst["input_ids"]:
    maximum = max(maximum, x.shape[0])
    minimum = min(minimum, x.shape[0])


In [39]:
torch.max(dst[140:150]["input_ids"])

tensor(51926)

In [80]:
%load_ext autoreload
%autoreload 2
from datamodules import NewsDataModule

In [83]:
mod = NewsDataModule("server", 6, "ufal/robeczech-base", ".cache", 512)

In [10]:
dst = dataset.select(range(100))

In [30]:
tokenizer = AutoTokenizer.from_pretrained("ufal/robeczech-base")

In [33]:
from transformers import RobertaTokenizerFast
tokenizer.bos_token

'[CLS]'

In [45]:
tokenizer("Ahoj, ahoj", truncation=True, max_length=2, add_special_tokens=False)

{'input_ids': [45487, 90], 'attention_mask': [1, 1]}

In [74]:
from transformers import AutoTokenizer
max_length = 512
trun_type = "end"

def truncate_tokenized(tokenized):
    if trun_type == "start":
        truncated = tokenized[:max_length]
    elif trun_type == "end":
        truncated = tokenized[-max_length:]
    # CLS and SEP tokens
    truncated[0] = tokenized[0]
    truncated[-1] = tokenized[-1]
    return truncated




def tokenize(examples):
    tokenizer = AutoTokenizer.from_pretrained("ufal/robeczech-base", use_fast=True)
    content = examples["content"]
    tokenized = tokenizer(content, truncation=False, padding=False)
    for key in ["input_ids", "attention_mask"]:
        tokenized[key] = [truncate_tokenized(x) for x in tokenized[key]]
    return tokenized

In [97]:
mod = NewsDataModule("server", 6, "ufal/robeczech-base", ".cache", 512, trunc_type="end")

In [100]:
dst = dst.map(mod.tokenize, batched=True, batch_size=5)

The OrderedVocab you are attempting to save contains a hole for index 51959, your vocabulary could be corrupted !


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [101]:
dataset = dst.filter(lambda batch: [x != 0 for x in batch["server"]], batched=True, keep_in_memory=True)

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

In [91]:
[len(x) for x in dst["input_ids"]]

[512,
 512,
 512,
 166,
 99,
 123,
 107,
 512,
 337,
 512,
 328,
 458,
 512,
 512,
 512,
 407,
 199,
 256,
 112,
 389,
 512,
 512,
 230,
 281,
 90,
 319,
 512,
 264,
 419,
 512,
 512,
 96,
 366,
 191,
 193,
 512,
 330,
 512,
 291,
 512,
 361,
 512,
 387,
 314,
 333,
 512,
 512,
 490,
 512,
 481,
 225,
 512,
 217,
 147,
 512,
 512,
 512,
 512,
 368,
 320,
 512,
 195,
 493,
 512,
 512,
 129,
 295,
 512,
 229,
 308,
 512,
 512,
 512,
 473,
 96,
 283,
 512,
 471,
 306,
 311,
 140,
 261,
 166,
 218,
 512,
 512,
 510,
 130,
 319,
 190,
 232,
 512,
 432,
 222,
 338,
 482,
 147,
 232,
 366,
 220]

In [20]:
maximum = 0
for x in dm.train_dataloader():
    maximum = max(maximum, torch.max(x["input_ids"]))
    assert torch.all(x["attention_mask"] == 5)

NameError: name 'dm' is not defined

In [ ]:

# Full Code:
import torch
import lightning as L
import torchmetrics
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from timm import create_model


class CIFAR10DataModule(L.LightningDataModule):
    
    def __init__(self, batch_size = 32) -> None:
        super().__init__()
        self.transform = transforms.Compose(
            [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
        )

        self.batch_size = batch_size

        self.train_set = torchvision.datasets.CIFAR10(
            root="~/data", train=True, download=True, transform=self.transform
        )
        self.test_set = torchvision.datasets.CIFAR10(
            root="~/data", train=False, download=True, transform=self.transform
        )
        
    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            self.train_set, batch_size=self.batch_size, shuffle=True, num_workers=4
        )
        
    def test_dataloader(self):
        return torch.utils.data.DataLoader(
            self.test_set, batch_size=self.batch_size, shuffle=False, num_workers=4
        )



class LitModel(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = torch.compile(
            create_model("resnet18", num_classes=10)
        )
        self.criterion = nn.CrossEntropyLoss()
        self.accuracy_score = torchmetrics.Accuracy(
            task="multiclass", num_classes=10
        )

    def forward(self, x):
        return self.model(x)

    def common_step(self, x, y, stage):
        logits = self.model(x)
        loss = self.criterion(logits, y)
        acc = self.accuracy_score(logits, y)
        self.log(
            f"{stage}_loss", loss, on_epoch=True, prog_bar=True, logger=True
        )
        self.log(
            f"{stage}_acc", acc, on_epoch=True, prog_bar=True, logger=True
        )
        return loss, acc

    def training_step(self, batch, batch_idx):
        x, y = batch
        loss, acc = self.common_step(x, y, "train")
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        loss, acc = self.common_step(x, y, "test")
        return {"test_loss": loss, "test_acc": acc}

    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=0.001)


def main():
    datamodule = CIFAR10DataModule()
    model = LitModel()
    trainer = L.Trainer(
        max_epochs=3,
        devices="auto",
    )
    trainer.fit(model, datamodule=datamodule)
    test_result = trainer.test(ckpt_path="best", datamodule=datamodule)
    print(test_result)

if __name__ == "__main__":
    main()

In [1]:
from datasets import load_dataset
dst = load_dataset("hynky/czech_news_dataset", "test")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/122056 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/122056 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1383298 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /home/kydliceh/.cache/huggingface/datasets/hynky___parquet/hynky--czech_news_dataset-0cbab0cb100c6b2f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:

keys = ["server", "authors_cum_gender", "category", "day_of_week"]
dst = dst.filter(lambda x: all([y != 0 for y in [x[key] for key in keys]]))

Loading cached processed dataset at /home/kydliceh/.cache/huggingface/datasets/hynky___parquet/hynky--czech_news_dataset-0cbab0cb100c6b2f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-03b5f3701586bbc9.arrow
Loading cached processed dataset at /home/kydliceh/.cache/huggingface/datasets/hynky___parquet/hynky--czech_news_dataset-0cbab0cb100c6b2f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-b92e720c79b8f82e.arrow
Loading cached processed dataset at /home/kydliceh/.cache/huggingface/datasets/hynky___parquet/hynky--czech_news_dataset-0cbab0cb100c6b2f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-2171c3bccd434849.arrow


In [16]:
lenghts = [len(x) for x in dst["train"]["content"]]

In [24]:
avg_len = sum(lenghts) / len(lenghts)

In [26]:
nchars_to_ntokens_approx(1400)

515

In [14]:
import math
def nchars_to_ntokens_approx(nchars):
    #returns an estimate of #tokens corresponding to #characters nchars
    return max(0,int((nchars)*math.exp(-1)))

In [30]:
dst["train"].features["authors_cum_gender"].int2str(0)

'None'

In [35]:
trans_gender ={
    "MAN": "Muž",
    "WOMAN": "Žena",
    "MIXED": "Smíšené pohlaví"
}
trans_server = {
    "denik": "Deník.cz",
    "idnes": "iDNES.cz",
    "seznamzpravy": "SeznamZprávy",
    "aktualne": "Aktuálně.cz",
    "novinky": "Novinky.cz",
    "irozhlas": "iRozhlas.cz",
}
trans_day_of_week = {
    "MONDAY": "Pondělí",
    "TUESDAY": "Úterý",
    "WEDNESDAY": "Středa",
    "THURSDAY": "Čtvrtek",
    "FRIDAY": "Pátek",
    "SATURDAY": "Sobota",
    "SUNDAY": "Neděle",
}

trans_category = {n: n for n in dst["train"].features["category"].names}
trans = {
    "authors_cum_gender": trans_gender,
    "category": trans_category,
    "day_of_week": trans_day_of_week,
    "server": trans_server
}

In [38]:
max_length = 1400
features = dst["train"].features
def convert_for_gpt3(examples):
    truncated_content = examples["content"][:max_length]
    keys = [ "server",  "category", "authors_cum_gender", "day_of_week"]
    retrieved_keys = {key: trans[key][features[key].int2str(examples[key])] for key in keys}
    return {
        "query": truncated_content,
        "completion": " ".join([value for value in retrieved_keys.values()]),
    }
convert_for_gpt3(dst["train"][0])





{'query': 'Telefon má spoustu zajímavých funkcí. Jednou z nich je například i funkce hlasového vytáčení a ovládání telefonu. Mobil má paměť na 15 hlasových povelů, které jsou volně sdílené mezi povely pro vytočení určitého čísla a dále mohou být přiřazeny až k 25 dalším funkcím (příkazům) telefonu. V souvislosti s hlasovými funkcemi je zde i paměť na 20 sekundovou hlasovou nahrávku, což není mnoho, ale pro občasnou potřebu to bohatě stačí. Hmotnostně patří Xenium spíše ke kolibříkům. Považte sami, 95 gramů ještě dnes nemá kdejaký telefon. Rozměry 109x41x20 mm jsou přijatelné celkem i pro kapsičku u košile. Další funkcí, kterou disponuje Xenium je vibrační zvonění, což není dnes již neobvyklé. Snad jen, kdyby se tento telefon zaměřil na střední cenovou třídu. Skládání vlastních melodií se dnes již vyžaduje také snad od každého mobilu. Tento přístroj editorem na melodie rovněž disponuje. Dále je zde tzv. Brick game, což my spektristé známe pod jménem Arkanoid. Je až s podivem, jak rychle

range(473013, 523013)

In [46]:
TRAIN_SIZE = 50000
TEST_SIZE = 10000
train_dst = dst["train"].select(range(len(dst["train"]) - TRAIN_SIZE, len(dst["train"])))
test_dst = dst["test"].shuffle().select(range(TEST_SIZE))



In [48]:
train_dst = train_dst.map(convert_for_gpt3)
test_dst = test_dst.map(convert_for_gpt3)


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [49]:
train_dst = train_dst.remove_columns(set(train_dst.column_names) - {"query", "completion"})
test_dst = test_dst.remove_columns(set(test_dst.column_names) - {"query", "completion"})

In [54]:
train_df = train_dst.to_pandas()
test_df = test_dst.to_pandas()
 


In [ ]:
train_df.to_json("gpt3-train.jsonl", orient='records', lines=True)
test_df.to_json("gpt3-test.jsonl", orient='records', lines=True)